In [1]:
from kfp.v2.dsl import component

@component(
    base_image="python:3.9-slim",
    packages_to_install=["minio", "kubernetes"]
)
def create_and_upload_op(username: str):
    import os
    from minio import Minio
    from pathlib import Path
    from kubernetes import client, config
    import base64

    # 1. Load in-cluster config using mounted ServiceAccount credentials
    config.load_incluster_config()

    # 2. Get namespace (from serviceaccount token mount)
    with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace", "r") as f:
        namespace = f.read().strip()

    # 3. Use Kubernetes API to get the secret
    secret_name = "minio-secret"
    v1 = client.CoreV1Api()
    secret = v1.read_namespaced_secret(secret_name, namespace)

    # 4. Decode the base64-encoded secret values
    access_key = base64.b64decode(secret.data["MINIO_ACCESS_KEY"]).decode()
    secret_key = base64.b64decode(secret.data["MINIO_SECRET_KEY"]).decode()
    endpoint = os.getenv("MINIO_ENDPOINT", "minio-service.kubeflow:9000")

    # 5. Write a test file
    local_file = "/tmp/hello.txt"
    Path(local_file).write_text(f"hello {username}")

    # 6. Upload to MinIO
    minio_client = Minio(
        endpoint,
        access_key=access_key,
        secret_key=secret_key,
        secure=False
    )

    bucket = "pipeline-artifacts"
    if not minio_client.bucket_exists(bucket):
        minio_client.make_bucket(bucket)

    minio_client.fput_object(bucket, "hello.txt", local_file)
    print(f"✅ Uploaded hello.txt to {bucket}/hello.txt on {endpoint}")


/tmp/ipykernel_320/3194792253.py:1: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  from kfp.v2.dsl import component


In [2]:
from kfp.v2.dsl import pipeline

@pipeline(name="minio-sa-secret-pipeline")
def artifact_pipeline(username: str):
    create_and_upload_op(username=username)


In [3]:
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=artifact_pipeline,
    package_path="minio_pipeline_sa_secret.yaml"
)
